# Fine-tuning BERT on CSI data with MLM

In [7]:
import torch
from torch.optim import AdamW
import pandas as pd
from glob import glob
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM
from tqdm.auto import tqdm

In [8]:
# load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# get path to all sub files
sub_files = glob('cleaned_subs/*.tsv')

# return all subs lines as list
def data_loader(files):
    'loads texts into a single list'
    texts = []
    for f in sub_files:
        df = pd.read_csv(f, sep='\t', usecols=['text'])
        texts += list(df["text"].values)
    return texts

data = data_loader(sub_files)
len(data)

216856

In [10]:
print(data[0])
print(data[-1])

My name is Royce Harmon.
You should watch it.


In [11]:
# tokenize the data with the BERT tokenizer
inputs = tokenizer(data, 
    max_length=512,
    truncation=True,
    padding=True,
    return_tensors='pt')

print(inputs.keys())
print(inputs['input_ids'].shape)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([216856, 91])


In [30]:
inputs.input_ids[0]

tensor([  101,  2026,  2171,  2003, 14789, 25546,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0])

In [12]:
# deep copy the inputs to labels (because we will mask the inputs)
inputs['labels'] = inputs['input_ids'].detach().clone()
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [13]:
# mask with 15% probability

# create random tensor(0-1) of size inputs
random_tensor = torch.rand(inputs['input_ids'].shape)
# create a mask with 15% selected while also ingnoring
# [cls] == 101, [sep] == 102, and [pad] == 0
masked_tensor = (random_tensor < 0.15)*(inputs['input_ids'] != 101)*(inputs['input_ids'] != 102)*(inputs['input_ids'] != 0)

# check sizes align
masked_tensor.shape, inputs['input_ids'].shape

(torch.Size([216856, 91]), torch.Size([216856, 91]))

In [14]:
# getting indices where masked is True
non_zero_indices = []
for mask in masked_tensor:
    non_zero_indices.append(torch.flatten(mask.nonzero()).tolist())

print(non_zero_indices[:20])

[[], [4, 13], [], [4, 6], [3, 17], [5], [5, 6], [9], [], [4], [3], [13], [5, 12, 17], [], [1, 5], [], [1], [5, 22, 24], [4], [1]]


In [15]:
# replace masked values with 103
for i in range(len(inputs['input_ids'])):
    inputs['input_ids'][i, non_zero_indices[i]] = 103

In [16]:
## pytorch dataset class

class CSIDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, index):
        input_ids = self.encodings['input_ids'][index]
        attention_mask = self.encodings['attention_mask'][index]
        token_type_ids = self.encodings['token_type_ids'][index]
        labels = self.encodings['labels'][index]
        return {
            'input_ids':input_ids, 
            'attention_mask':attention_mask,
            'token_type_ids':token_type_ids,
            'labels': labels
        }


In [17]:
# create torch dataset
dataset = CSIDataset(inputs)

In [18]:
# create torch data loader
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=True
    )

In [19]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [20]:
model.to(device);

In [21]:
epochs = 32
optimizer = AdamW(model.parameters(), lr=1e-5)

model.train();


In [22]:

# training loop
for epoch in range(epochs):
    loop = tqdm(data_loader)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        # print(input_ids.shape)
        labels = batch['labels']
        attention_mask =  batch['attention_mask']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        loop.set_description(f'Epoch: {epoch}')
        loop.set_postfix(loss=loss.item())


Epoch: 31: 100%|██████████| 1/1 [00:05<00:00,  5.38s/it, loss=5.42]


In [23]:
model.save_pretrained('model/bert-base-uncased-csi-trained');


In [24]:
# model = BertForMaskedLM.from_pretrained("bert-base-uncased");
model.from_pretrained('model/bert-base-uncased-csi-trained');


def predict_masked_sent(text, top_k=5):
    # Tokenize input
    text = f"[CLS] {text} [SEP]"
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("\t[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

        
predict_masked_sent("You should [MASK] it.", top_k=5)


	[MASK]: 'see'  | weights: 0.12207724153995514
	[MASK]: 'take'  | weights: 0.0773620456457138
	[MASK]: 'have'  | weights: 0.06554163992404938
	[MASK]: 'find'  | weights: 0.049902647733688354
	[MASK]: 'try'  | weights: 0.047774046659469604


In [25]:
import random
for sent in random.sample(data, 5):
    print(sent)
    words = sent.split(' ')
    mask_int = np.random.randint(0,len(words),1)
    words[mask_int[0]] = '[MASK]'
    masked_sent = ' '.join(words)
    print(masked_sent)
    predict_masked_sent(masked_sent, top_k=5)


I'll be up here.
[MASK] be up here.
	[MASK]: 'should'  | weights: 0.6194536685943604
	[MASK]: 'gotta'  | weights: 0.14924505352973938
	[MASK]: 'must'  | weights: 0.09466520696878433
	[MASK]: 'could'  | weights: 0.01983913965523243
	[MASK]: 'please'  | weights: 0.01004601176828146
We aren't going to lunch are we?
We aren't going to lunch [MASK] we?
	[MASK]: 'are'  | weights: 0.9971518516540527
	[MASK]: 'aren'  | weights: 0.0013718613190576434
	[MASK]: 'were'  | weights: 0.0012275198241695762
	[MASK]: 'have'  | weights: 0.0001236569951288402
	[MASK]: 'is'  | weights: 4.580666427500546e-05
Belladonna would help explain it.
[MASK] would help explain it.
	[MASK]: 'that'  | weights: 0.29083457589149475
	[MASK]: 'it'  | weights: 0.18923956155776978
	[MASK]: 'nothing'  | weights: 0.03527429699897766
	[MASK]: 'this'  | weights: 0.023704886436462402
	[MASK]: 'he'  | weights: 0.023436715826392174
Thanks. You can get dressed now.
Thanks. You can [MASK] dressed now.
	[MASK]: 'get'  | weights: 0.939

## Getting the sentence embedding

In [40]:
import torch
# import torch.nn.functional as F
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model.from_pretrained('model/bert-base-uncased-csi-trained');

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def sentence_embedder(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings


sentences = data[:5]
sentence_embeddings = sentence_embedder(sentences)
print("Sentence embeddings:")
print(sentence_embeddings)




Sentence embeddings:
tensor([[-0.0021, -0.0029, -0.0034,  ..., -0.0028, -0.0051, -0.0058],
        [ 0.0016,  0.0005, -0.0003,  ..., -0.0010, -0.0011, -0.0059],
        [-0.0026, -0.0040, -0.0042,  ..., -0.0047, -0.0042, -0.0050],
        [-0.0044, -0.0052, -0.0054,  ..., -0.0049, -0.0042, -0.0070],
        [-0.0036, -0.0045, -0.0045,  ..., -0.0037, -0.0040, -0.0052]])


In [41]:
sentence_embeddings[0].shape

torch.Size([30522])